In [5]:
from figure_utils import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
SAVE = False
runtime_df = pd.read_parquet(f"{RESULT_DIR}/daic/preprocessed.parquet")
gpu_mapping = {'2080': '2080Ti', '1080': '1080Ti', 'a40': 'A40', 'p100': 'P100', 'v100': 'V100'}
runtime_df = runtime_df[
    # (runtime_df.dataset_type == 'synthetic') &
    (~runtime_df.operator.isin(['Noop', 'Materialization']))
    ]

runtime_df.compute_unit = runtime_df.compute_unit.astype(str).apply(lambda x: gpu_mapping.get(x, x))
runtime_df['compute_unit'].unique()

KeyboardInterrupt: 

In [ ]:
runtime_df['time_saved'] = runtime_df.times_mean * runtime_df.speedup - runtime_df.times_mean
runtime_df[runtime_df.label][['dataset', 'speedup', 'operator', 'compute_unit', 'times_mean', 'time_saved']].sort_values('time_saved', ascending=False).head(10)

,dataset,speedup,operator,compute_unit,times_mean,time_saved
346000,movie,8.339612,Linear Regression,P100,3.304723,24.255383
345712,movie,8.199133,Linear Regression,P100,3.071785,22.114189
344704,expedia,5.131083,Linear Regression,P100,5.139184,21.230399
344416,expedia,6.250318,Linear Regression,P100,3.905428,20.504739
344560,expedia,4.868123,Linear Regression,P100,5.114525,19.783613
345856,movie,6.685368,Linear Regression,P100,3.149884,17.908252
344414,expedia,6.886286,Linear Regression,1080Ti,2.714341,15.977386
345998,movie,9.284375,Linear Regression,1080Ti,1.906042,15.790362
345710,movie,9.276487,Linear Regression,1080Ti,1.906777,15.781411
344702,expedia,5.326763,Linear Regression,1080Ti,3.506220,15.170583


In [8]:
import numpy as np
from scipy import stats
from itertools import combinations

def ttest_run(c1, c2):
    results = stats.ttest_ind(runtime_df[runtime_df.compute_unit == c1].times_mean, runtime_df[runtime_df.compute_unit == c2].times_mean, equal_var = False)
    df = pd.DataFrame({'categ1': c1,
                       'categ2': c2,
                       'tstat': results.statistic,
                       'pvalue': results.pvalue}, 
                       index = [0])    
    return df

df_list = [ttest_run(i, j) for i, j in combinations(runtime_df['compute_unit'].unique().tolist(), 2)]

final_df = pd.concat(df_list, ignore_index = True)
final_df.sort_values('pvalue')


,categ1,categ2,tstat,pvalue
0,CPU 08c,CPU 32c,50.884896,0.000000e+00
17,CPU 16c,2080,58.814473,0.000000e+00
16,CPU 16c,a40,57.347293,0.000000e+00
15,CPU 16c,p100,46.069399,0.000000e+00
14,CPU 16c,v100,57.166953,0.000000e+00
12,CPU 32c,2080,38.654197,0.000000e+00
13,CPU 16c,1080,52.115470,0.000000e+00
6,CPU 08c,2080,69.546845,0.000000e+00
2,CPU 08c,1080,65.578054,0.000000e+00
5,CPU 08c,a40,68.689220,0.000000e+00
